In [3]:
from chromalab.observer import Observer
from chromalab.spectra import Spectra, Illuminant
import colour
from colour.models import RGB_COLOURSPACE_sRGB
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from tqdm import tqdm
%matplotlib widget

In [100]:
def getSpectralLocusMS(wavelengths, responseFunctions, cmfs, transtype="stockmansharpe"):
    """
    Let n be the size of the array, wavelengths, and d be the size of responseFunctions.
    i.e. n is the number of wavelengths we are sampling at. d is the dimension of the color solid.
    Input:
    responseFunctions - np.array([coneresponse_1, conresponse_2, ... coneresponse_d]) has shape d, n
        coneresponse_i has n values between 0 and 1, describing the amount of response at each wavelength in wavelengths
    Output:
    Returns an ndarray of shape (n, d) describing the spectral locus, defined in page 6 of jessicas paper
    and also the corresponsing rgb colors
    """
    responseFunctionsTemp = np.dstack([np.zeros(responseFunctions.shape[1]), responseFunctions[0], responseFunctions[1]])[0]
   
    # responseColors = [lms_to_rgb(lms) for lms in responseFunctionsTemp.T] # later on change to true colors
    # responseColors = [wavelength_to_rgb(wavelength) for wavelength in wavelengths]
    xyzs = ms2xyz(wavelengths, responseFunctionsTemp)
    responseColors = [xyz_to_rgb(xyz, transtype) for xyz in xyzs]
    return responseFunctions.T, responseColors

def wavelength_to_rgb(wavelength, transtype="stockmansharpe"):
    """
    convert wavelength in nm to standard ciexyz to rgb d65 illuminant (NOT BASED OFF OF ANY CONE CELL STUFF)
    """
    xyz = colour.colorimetry.wavelength_to_XYZ(wavelength)
    return xyz_to_rgb(xyz, transtype)

def plot2DLocus(points, wavelengths, colors,
                title="MS Spectral Locus",
                xlabel="M Cone Response", 
                ylabel="S Cone Response"):

    plt.close()
    plt.plot(points[:,0], points[:,1], 'b-', zorder=0)
    scatter = plt.scatter(points[:,0], points[:,1], c=colors, zorder=1)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

def xyz_to_rgb(xyz, transtype="stockmansharpe"):
    """
    xyz to rgb under the d65 illuminant
    """
    if transtype == "stockmansharpe":
        T = np.array([[3.2406, -1.5372, -0.4986],
                      [-0.9689, 1.8758, 0.0415],
                      [0.0557, -0.2040, 1.0570]])
    elif transtype=="bradford":
    # bradfords spectrally sharpened
        T = np.array([[0.8951, 0.2664, -0.1614],
                     [-0.7502, 1.7135, 0.0367],
                     [0.0389, -0.0685, 1.0296]])
    elif transtype == "97":
        T = np.array([[0.8562, 0.3372, -0.1934],
                      [-0.8360, 1.8327, 0.0033],
                      [0.0357, -0.0469, 1.0112]])
    elif transtype == "02":
        T = np.array([
            [0.7328, 0.4296, -0.1624],
            [-0.7036, 1.6975, 0.0061],
            [0.0030, 0.0136, 0.9834]
        ])
    elif transtype == "16":
        T = np.array([
            [0.401288, 0.650173, -0.051461],
            [-0.250268, 1.204414, 0.045854],
            [-0.002079, 0.048952, 0.953127]
        ])
        
    # for D65, linear relationship
    
    return gamma_correct(np.clip(T @ xyz, 0, 1))    

def ms2xyz(wavelengths, cmfs, msresponses):
    """
    the wavelengths in the cmf should match up with that in lmsresponses
    """
    xyzcolors = []
    A = np.array([[0.68990272, 0.34832189, 0],
                  [0.0, 0.0, 1.93]])
    print(cmfs.T @ cmfs)
    A_plus = np.linalg.inv(cmfs.T @ cmfs) @ cmfs.T # moore penrose pseudoinverse
    xyzcolors = msresponses @ A_plus
    return np.array(xyzcolors)

def gamma_correct(rgb):
    rgb_corrected = np.where(
        rgb <= 0.0031308,
        12.92 * rgb,
        1.055 * np.power(rgb, 1 / 2.4) - 0.055
    )
    return rgb_corrected

In [101]:
df = pd.read_csv('data/lin2012xyz2e_5_7sf.csv') # lms to xyz color mathcing function

# Display the first few rows of the dataframe
wavelengths = df.iloc[:, 0].to_numpy()
cmf1 = df.iloc[:, 1].to_numpy()
cmf2 = df.iloc[:, 2].to_numpy()
cmf3 = df.iloc[:, 3].to_numpy()
cmfs = np.dstack([cmf1, cmf2, cmf3])[0]

In [102]:
# loading data for 2D case
standard_dichromat = Observer.dichromat(wavelengths)
ms_responses = np.vstack((standard_dichromat.sensors[1].data, # m response
                          standard_dichromat.sensors[0].data,  # s response
))
mspoints, mscolors = getSpectralLocusMS(wavelengths, ms_responses, cmfs)
plot2DLocus(mspoints, wavelengths, mscolors, title="MS Spectral Locus from CIE 10-deg CMFs \n and 02 XYZ to RGB linear transformation",
                xlabel="M Cone Response", 
                ylabel="S Cone Response")


[[ 0.47596576  0.24030822  0.        ]
 [ 0.24030822  0.12132814  0.        ]
 [ 0.          0.          3.7249    ]]


LinAlgError: Singular matrix